In [15]:
import os
from os import listdir
from os.path import isdir, join, exists
import shutil
import glob
import subprocess
import re
import json
import contextlib

In [16]:
###############################################################################
# REMatcher is a utility class to make regex's work similar to Perl
# NOTE: if there are any changes needed in this function contact Sean.
###############################################################################
class REMatcher(object):
    ''' This is a utility function to help out with matching regex's and
        grabbing the matches out in a way that is similar to how perl
        does it.
    '''
    def __init__(self, matchstring: str) -> None:
        self.matchstring = matchstring

    def match(self, regexp: str) -> bool:
        self.rematch = re.match(regexp, self.matchstring,
                                re.IGNORECASE |
                                re.MULTILINE |
                                re.DOTALL
                                )
        return bool(self.rematch)

    def group(self, i: int) -> str:
        return "Error 999" if self.rematch is None else self.rematch.group(i)

In [17]:
def create_json_file_from_dirs(path, json_file_name, repo_root):
    test_list = []

    for dir_name in next(os.walk(path))[1]:
        test_path = os.path.join(path, dir_name)

        # Calculate the relative path after the repo root
        test_name = os.path.relpath(test_path, repo_root)

        test_dict = {"test": test_name}
        param_dict = {}

        # Read the Makefile
        with contextlib.suppress(FileNotFoundError):
            with open(os.path.join(test_path, "Makefile"), "r") as f:
                lines = f.readlines()

                # Parse for COMPILE_ARGS
                for line in lines:
                    if re.match(r"^\s*#", line):
                        continue  # Skip comments

                    compile_args_match = re.findall(r'-P\s*\$\(DUT\)\.(\w+)=(\w+)', line)
                    for param, value in compile_args_match:
                        param_dict[param] = value

                # Parse for SEED
                for line in lines:
                    if re.match(r"^\s*#", line):
                        continue  # Skip comments

                    if seed_match := re.search(r'export SEED=(\w+)', line):
                        test_dict["seed"] = seed_match.group(1)

        test_dict["param"] = param_dict
        test_list.append(test_dict)

    # Sort the list of dictionaries by the 'test' key
    test_list.sort(key=lambda x: x['test'])

    # Create the JSON file
    with open(json_file_name, 'w') as f:
        json.dump(test_list, f, indent=4)

In [18]:
def get_unique_dir_name(base_name):
    counter = 0
    new_name = base_name
    while os.path.exists(new_name):
        new_name = f'{base_name}.{counter}'
        counter += 1
    return new_name

In [19]:
def process_results(output_file):
    with open(output_file, 'r') as f:
        line_list = [line.strip() for line in f.readlines()]

    for line in line_list:
        m = REMatcher(line)
        if m.match(r'.*TESTS=(\d+) PASS=(\d+) FAIL=(\d+) SKIP=(\d+).*'):
            tests_tot = int(m.group(1))
            tests_pass = int(m.group(2))
            tests_fail = int(m.group(3))
            tests_skip = int(m.group(4))
            print(f'found the results line... {tests_tot=} {tests_pass=} {tests_fail=} {tests_skip=}')
            return tests_fail <= 0
    return False


In [20]:
def update_makefile(makefile_path: str, seed: str, params: dict):
    new_lines = []
    with open(makefile_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "COMPILE_ARGS" in line and "export SEED" not in line:
                # Remove existing parameters
                if not params:
                    continue
            
            if "export SEED" in line and seed is not None:
                line = f"export SEED={seed}\n"

            new_lines.append(line)
        
    if params:
        param_str = " ".join([f"-P $(DUT).{k}={v}" for k, v in params.items()])
        new_lines.append(f"COMPILE_ARGS += {param_str}\n")
    
    with open(makefile_path, 'w') as f:
        f.writelines(new_lines)


In [21]:
def run_make(repo_root: str, test: str, base_test_path: str, path_out: str, env, seed: str, params: dict):
    # Build a custom folder name based on the seed and parameters
    folder_suffix = f"seed_{seed}"
    for param, value in params.items():
        folder_suffix += f"_{param}_{value}"
    
    # Combine the test name and custom folder suffix
    custom_test_folder = f"{test}_{folder_suffix}"
    
    # Update the test path
    regression_test_path = os.path.join(path_out, custom_test_folder)

    # set up some local variables
    my_command = 'make'
    rpt = 'results.txt'
    output_file = f'{regression_test_path}/{rpt}'

    test_path = f'{repo_root}/{base_test_path}'
    if os.path.exists(test_path):
        shutil.copytree(test_path, regression_test_path)
        
    pycache = f'{regression_test_path}/__pycache__/'
    if os.path.exists(pycache):
        shutil.rmtree(pycache)

    os.chdir(f'{regression_test_path}')
    
    # Update Makefile based on the provided seed and params
    makefile_path = f'{regression_test_path}/Makefile'
    update_makefile(makefile_path, seed, params)

    # Execute the command and capture both stdout and stderr
    try:
        completed_process = subprocess.run('make clean', shell=True, check=True, capture_output=True, text=True)
        completed_process = subprocess.run(my_command, env=env, shell=True, check=True, capture_output=True, text=True)
        output_text = completed_process.stdout

        # Save the output to a file
        with open(output_file, 'w') as file:
            file.write(output_text)

        # print("Command executed successfully. Output saved to:", output_file)
        pass_or_fail = process_results(output_file)

    except subprocess.CalledProcessError as e:
        print(f'Error with {test=}')
        print("Command failed with exit code:", e.returncode)
        print("Error message:", e.stderr)
        # Save the output to a file
        with open(output_file, 'w') as file:
            file.write(e.stderr)
        pass_or_fail = False
    return pass_or_fail

In [22]:
# def remove_all_files(directory_path):
#     for filename in os.listdir(directory_path):
#         file_path = os.path.join(directory_path, filename)
#         try:
#             if os.path.isfile(file_path):
#                 os.unlink(file_path)
#         except Exception as e:
#             print(f'Failed to delete {file_path}. Reason: {e}')

In [23]:
# Save the current directory
original_directory = os.getcwd()
# get the repo root
repo_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')
print(f'{repo_root=}')
env = os.environ.copy()
env['REPO_ROOT'] = repo_root
config_file = f'{repo_root}/bin/config.json'
with open(config_file, 'r') as f:
    config_dct = json.load(f)
test_list = 'level0'
tag = 'l0_run'


repo_root='/home/sean/github/RTL_Design_Projects'


In [24]:
# create_json_file_from_dirs(f"{repo_root}/val/common_cocotb_only", f"{repo_root}/val/testlists/level0.json", repo_root)

In [25]:
regression_dir = f'{repo_root}/regression/{tag}'
regression_dir = get_unique_dir_name(regression_dir)
os.mkdir(regression_dir)
cleanall = f'{repo_root}/{config_dct["make_clean"]}'
shutil.copy(cleanall, regression_dir)


'/home/sean/github/RTL_Design_Projects/regression/l0_run/cleanall.mk'

In [26]:
try:
    json_file_path = f"{repo_root}/{config_dct['test_lists'][test_list]}"
except KeyError:
    print(f'Unknown test list {test_list}')
    exit(-1)

# Read JSON file
with open(json_file_path, 'r') as f:
    test_list_json = json.load(f)

fail_count = 0
fail_list = []
for test_count, test_entry in enumerate(test_list_json):
    test_path = test_entry['test']
    test = test_path.split('/')[-1]
    if len(test) == 0:
        test = test_path.split('/')[-2]

    seed = test_entry.get('seed', 1234)   # Default to None if 'seed' is not in dict
    params = test_entry.get('param', {})  # Default to empty dict if 'param' is not in dict

    pass_or_fail = run_make(repo_root, test, test_path, regression_dir, env, seed, params)
    if pass_or_fail is False:
        fail_count += 1
        fail_list.append(test)

# Change back to the original directory
os.chdir(original_directory)

found the results line... tests_tot=2 tests_pass=2 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0
found the results line... tests_tot=1 tests_pass=1 tests_fail=0 tests_skip=0

In [27]:
report_str = f'''
======================================================================
 Test Count = {test_count+1}        Failures = {fail_count}
======================================================================
Failure List:
'''
for item in fail_list:
    report_str += f'    {item}\n'
print(report_str)


 Test Count = 42        Failures = 0
Failure List:



In [28]:
with open(f'{regression_dir}/report.txt', 'w') as file:
    file.write(report_str)